In [ ]:
import pandas as pd
import os
import glob
import concurrent.futures
import time

directory = 'C:\\Users\\minhtriet.pham\\Desktop\\Data Invoice 20230327\\NAM 2023\\Month 2'
directory2 = 'C:\\Users\\minhtriet.pham\\Desktop\\Data Invoice 20230327\\NAM 2023\\Month 2'
cate_dir = 'C:\\Users\\minhtriet.pham\\Desktop\\DataCate.xlsx'
cate_dir2 = 'C:\\Users\\minhtriet.pham\\Desktop\\DataCate.xlsx'

# list of Excel files to read data from
if os.path.exists(os.path.join(directory, '230201 Payment Summary - 2023-01-31T145747.301.xlsx')):
    excel_files = glob.glob(os.path.join(directory, '*.xlsx'))
else:
    excel_files = glob.glob(os.path.join(directory2, '*.xlsx'))
try:
    # use cols 0, Barcode; 2, Category Name
    # df2 = pd.read_excel(cate_dir, sheet_name='Sheet1', usecols=[0,2])
    # use cols 0, Barcode; 3, SubCategory Name
    df2 = pd.read_excel(cate_dir, sheet_name='Sheet1', usecols=[0,3])
    pass
except FileNotFoundError:
    # df2 = pd.read_excel(cate_dir2, sheet_name='Sheet1', usecols=[0,2])
    # use cols 0, Barcode; 3, SubCategory Name
    df2 = pd.read_excel(cate_dir2, sheet_name='Sheet1', usecols=[0,3])
    pass

outputfile =  'dataM1.csv'
# def excel2csv
def excel2csv(inputfile, outputfile):
    
    # Check if exist file_path
    if not os.path.exists(outputfile):
        with open(outputfile, 'w') as f:
            pass

    # Read the first line of the file
    with open(outputfile, 'r') as f:
        first_line = f.readline().strip()
    # Check if the first line contains the expected column names
    has_headers = 'InvoiceID' in first_line
    df = pd.read_excel(inputfile, sheet_name='Sheet2', skiprows=2, usecols=[4,7,8,14])
    # df2 Category
    
    # merge with Category, on "Barcode"
    df = df.merge(df2, on='Barcode', how='left')
    
    # Check if the first line contains the expected column names
    df.to_csv(outputfile, mode='a', header= not has_headers, index=False)


# loop through the list of Excel files
with concurrent.futures.ThreadPoolExecutor() as executor:
    [executor.submit(excel2csv, file, outputfile) for file in excel_files]
    # for file in excel_files:
    #     executor.submit(excel2csv,file)

In [3]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

# Sample data
dtypes = {'InvoiceID': str, 'Barcode': str, 'Sum of Qty': str, 'Total Exclude VAT': str, 'SubCategory Name': str}
df = pd.read_csv('dataM1.csv', dtype=dtypes)

df['Sum of Qty'] = pd.to_numeric(df['Sum of Qty'], errors='coerce').fillna(0).astype(int)
df['Total Exclude VAT'] = pd.to_numeric(df['Total Exclude VAT'], errors='coerce').fillna(0).astype(float)
df['SubCategory Name'] = df['SubCategory Name'].astype(str)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 333854 entries, 0 to 333853
Data columns (total 5 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   InvoiceID          333854 non-null  object 
 1   Barcode            333854 non-null  object 
 2   Sum of Qty         333854 non-null  int64  
 3   Total Exclude VAT  333854 non-null  float64
 4   SubCategory Name   333854 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 12.7+ MB


In [4]:
# bill0 = df1.groupby('InvoiceID').filter(lambda x: x['Sum of Qty'].sum() == 0)

bill0 = df[df['Sum of Qty'] == 0]

bill0


,InvoiceID,Barcode,Sum of Qty,Total Exclude VAT,SubCategory Name
51,VN0001010201230023,260501000011,0,0.0,Hot pot Thai
119,VN0001010201230057,8936127796217,0,0.0,Sugar Tea
120,VN0001010201230057,8936127796019,0,0.0,Sugar Tea
262,VN0001010201230116,2010809000482,0,0.0,Cold Local Drinks
378,VN0001010201230168,8935049510864,0,0.0,Purified water
...,...,...,...,...,...
333454,VN0236010101230192,8934822120214,0,0.0,Import Beer Bottle
333656,VN0236010101230264,2701020000053,0,0.0,Onigiri
333674,VN0236010101230267,8938526588028,0,0.0,Dessert Sub
333725,VN0236010101230287,8934803026078,0,0.0,Salty Snacks


In [6]:
import statistics
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# filter out rows where 'Sum of Qty' or 'Total Exclude VAT' are zero
df_filtered = df[(df['Sum of Qty'] > 0) & (df['Total Exclude VAT'] > 0)]


# Calculate the IQR and filter out outliers
# Q1 = np.percentile(df_filtered['Total Exclude VAT'], 5)
# Q3 = np.percentile(df_filtered['Total Exclude VAT'], 95)
# IQR = Q3 - Q1
# upper_bound = Q3 + 1.5 * IQR
# lower_bound = 1000
# df_filtered = df_filtered[df_filtered['Total Exclude VAT'] <= upper_bound]

new_df = df_filtered.dropna().groupby('InvoiceID').agg({'Barcode': lambda x: ', '.join(x.astype(str)), 
                                      'Sum of Qty': 'sum', 
                                      'Total Exclude VAT': 'sum', 
                                      'SubCategory Name': lambda x: ', '.join(x.astype(str))})

new_df

<bound method NDFrame._add_numeric_operations.<locals>.mean of InvoiceID
VN0001010101230001      6481.0
VN0001010101230002     11112.0
VN0001010101230003    106482.0
VN0001010101230004    174478.0
VN0001010101230005     24074.0
                        ...   
VN0236020201230042     19091.0
VN0236020201230043    310216.0
VN0236020201230044     53290.0
VN0236020201230045    176363.0
VN0236020201230046     53637.0
Name: Total Exclude VAT, Length: 136439, dtype: float64>

In [25]:
# Calculate the statistical measures using describe() method
stat = new_df.describe()

# Extract the desired values from the summary
mean = stat.loc['mean']
# mode = stat.loc['mode']  # mode() returns a Series, we take the first element
std = stat.loc['std']
min_val = stat.loc['min']
max_val = stat.loc['max']

# Print the results
print("Mean:", mean)
# print("Mode:", mode)
print("Standard Deviation:", std)
print("Minimum Value:", min_val)
print("Maximum Value:", max_val)

print("Mode of Total Exculde VAT", new_df['Total Exclude VAT'].mode()[0])

Mean: Sum of Qty               3.000352
Total Exclude VAT    55431.369557
Name: mean, dtype: float64
Standard Deviation: Sum of Qty               3.186261
Total Exclude VAT    78693.171714
Name: std, dtype: float64
Minimum Value: Sum of Qty           1.0
Total Exclude VAT    1.0
Name: min, dtype: float64
Maximum Value: Sum of Qty               312.0
Total Exclude VAT    8443636.0
Name: max, dtype: float64
Mode of Total Exculde VAT 10909.0


In [ ]:
new_df.to_csv("dataCleanM2.csv")

In [42]:

# # Group the data based on Total Exclude VAT and calculate the counts
bins = np.concatenate(([0], np.arange(5000, 100001, 5000), [np.inf]))
labels = ['<5,000'] + ['{} to {}' .format(i, i+4999) for i in range(5000, 100000, 5000)] + ['>=100,000']
df_filtered['VAT_group'] = pd.cut(df_filtered['Total Exclude VAT'], bins=bins, labels=labels)

grouped_data = df_filtered.groupby('VAT_group')['Total Exclude VAT'].agg(['count','mean'])

grouped_data


/var/folders/dv/qwspr4sx6qjfkcywhq4tdqg00000gn/T/ipykernel_13903/2750019862.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['VAT_group'] = pd.cut(df_filtered['Total Exclude VAT'], bins=bins, labels=labels)


,count,mean
VAT_group,,
"<5,000",7301,3189.958225
5000 to 9999,60472,7974.530857
10000 to 14999,78583,12121.404757
15000 to 19999,52338,17646.044652
20000 to 24999,35035,22699.025746
25000 to 29999,24054,27458.379480
30000 to 34999,19580,32606.750306
35000 to 39999,13663,36987.820903
40000 to 44999,6045,42605.324235


In [44]:
grouped_data.to_csv('groupdata.csv')